In [2]:
import requests
from bs4 import BeautifulSoup
from lxml import etree
import json
import re
import multiprocessing
# import xlwt
import time
import pandas as pd

In [10]:
#总分榜单
jinjiang = 'https://www.jjwxc.net/topten.php?orderstr=7&t=1'
#'https://www.jjwxc.net/bookbase_slave.php?t=0&booktype=&opt=&page=1&endstr=true&orderstr=4'

header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                        'AppleWebKit/537.36 (KHTML, like Gecko) '
                        'Chrome/92.0.4515.107 Safari/537.36 Edg/92.0.902.55'}
response = requests.get(jinjiang, headers=header)
print(response.status_code)

200


In [11]:
# change to Chinese
response.encoding = response.apparent_encoding

print(response.text)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
    <head>
        <meta http-equiv="Content-Type" content="text/html; charset=gb18030"/>
        <meta http-equiv="X-UA-Compatible" content="IE=EmulateIE7"/>
                    <meta name="description" content="总分排行榜_晋江文学城
晋江文学城总分排行榜,是网站小说作品总积分排行榜。"/>
            <meta name="keywords" content="晋江文学城，作品总分排行，言情小说排行，纯爱小说排行，衍生小说排行，轻小说排行">
            <title>晋江文学城[总分排行榜]_按规定时间内作品积分排序</title>
                <link href="//s9-static.jjwxc.net/css/index.css" rel="stylesheet" type="text/css"/>
        <style>
            .current {font-weight: bold;color: red;}
        </style>
    </head>
    <body topmargin="0">
                        <link href="//static.jjwxc.net/css/channel_2010/index2010.css?ver=2022070803"
      rel="stylesheet"
      type="text/css"
      media="screen"/>
<style>
    #showNovelTro {position: absolute;dis

In [17]:
#Xpath
htmls = etree.HTML(response.text)
book_name = htmls.xpath('//a[@class="tooltip"]/text()')
print(book_name)

['全球高考', '某某', '破云', '破云2吞海', '判官', '死亡万花筒', '我行让我来[电竞]', 'AWM[绝地求生]', '烈火浇愁', '不见上仙三百年', '我在无限游戏里封神', 'FOG[电竞]', '地球上线', '嚣张', '不要在垃圾桶里捡男朋友[快穿]', '轻狂', '营业悖论[娱乐圈]', '将进酒', '默读', '当年万里觅封侯', '小蘑菇', '迪奥先生', '解药', '兼职无常后我红了', '我五行缺你', '放学等我', '欢迎进入梦魇直播间', '我开动物园那些年', '网恋翻车指南', '一个钢镚儿', '我只喜欢你的人设[娱乐圈]', '剑名不奈何', '我喜欢你的信息素', '残次品', '一级律师[星际]', '非职业半仙', '幻想农场', '三伏', '全能中单（电竞）', '可爱过敏原', '无限旅游团', '杀破狼', '骷髅幻戏图', '这该死的求生欲[穿书]', '空降热搜[娱乐圈]', '碎玉投珠', '无限练习生', '跨界演员', '不死者', '灵异片演员app[无限]', '图灵密码', '铜钱龛世', '我喜欢你男朋友很久了', '探虚陵现代篇', 'PUBG世纪网恋', '两A相逢必有一O', '影帝', '别来无恙', '定海浮生录', '万有引力[无限流]', '犯罪心理', '我死对头终于破产了', '为了和谐而奋斗', '太岁', '快穿之完美命运', '我爱种田', '夺梦', '星卡大师（重生）', 'C语言修仙', '黑天', '情敌每天都在变美[穿书]', '道医', '子夜十', '山海高中', '天宝伏妖录', '危险人格', '卡牌密室（重生）', '荣光[电竞]', '六爻', '古董下山', '新世纪，新冥界', '学神在手，天下我有', '魔尊他念念不忘', '全球进化后我站在食物链顶端', '没钱', '仙道第一小白脸', '沙雕渣攻今天又渣了我', '提灯映桃花', '我要这盛世美颜有何用', '熔城', '演技派', '快穿之打脸狂魔', '子夜鸮', '全世界都在等我们分手', '薄雾［无限］', '拯救悲情反派进行时！', '该我上场带飞了[全息]', '小行星', '我靠美颜稳住天下', '寒剑栖桃花', '三嫁咸鱼', '魔尊也想知道'

In [12]:
bs = BeautifulSoup(response.text, 'lxml')
print(bs)
# 放弃了 使用正则表达式吧。。。。
# find time
# [0-9]+[^%&',;=?$\x22]+[0-9]
# find author



<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=EmulateIE7" http-equiv="X-UA-Compatible"/>
<meta content="总分排行榜_晋江文学城
晋江文学城总分排行榜,是网站小说作品总积分排行榜。" name="description"/>
<meta content="晋江文学城，作品总分排行，言情小说排行，纯爱小说排行，衍生小说排行，轻小说排行" name="keywords"/>
<title>晋江文学城[总分排行榜]_按规定时间内作品积分排序</title>
<link href="//s9-static.jjwxc.net/css/index.css" rel="stylesheet" type="text/css"/>
<style>
            .current {font-weight: bold;color: red;}
        </style>
</head>
<body topmargin="0">
<link href="//static.jjwxc.net/css/channel_2010/index2010.css?ver=2022070803" media="screen" rel="stylesheet" type="text/css"/>
<style>
    #showNovelTro {position: absolute;display: none;padding: 4px;border: 1px solid #009900;background-color: #FFFFFF;width: 240px;z-index: 9999;}
    #showNovelTro ul {list-style-t

In [22]:
print(bs.findAll('tr')[5]) # 江停

<tr bgcolor="#eefaee" onmouseout="this.bgColor = '#eefaee';" onmouseover="this.bgColor = '#ffffff';">
<td align="center" height="23">3</td>
<td height="23"> <a href="oneauthor.php?authorid=294665" title="淮上">淮上</a>
</td>
<td height="23"> <a class="tooltip" href="onebook.php?novelid=3395943" rel="&lt;br&gt;城市天空，诡云奔涌&lt;br&gt;三年前恭州市的缉毒行动中，因总指挥江停判断失误，现场发生连环爆炸，禁毒支队伤亡惨重。三年后，本应早已因过殉职并尸骨无存的江停，竟奇迹般从植物人状态下醒来了。&lt;br&gt;英魂不得安息，他必须从地狱重返人间，倾其所有来还原血腥离奇的真相。&lt;br&gt;在本文人设中，严峫表面痞段子手但心细如丝且非常正气，江停智商很高身体素质很差，说话反复斟酌性格克制谨慎，表面温文儒雅但行事作风带邪性。两个主角人设清晰自洽，没有任何原型，不可作为任何代餐。&lt;br&gt;现代都市刑侦，英俊潇洒十项全能进可百米狙人头退可徒..." target="_blank" title="破云">破云</a>
</td>
<td align="center" height="23">
                                        原创-纯爱-近代现代-悬疑                                    </td>
<td align="center">正剧                                    </td>
<td align="center" height="23">
<font color="red">完结</font> </td>
<td align="right" height="23">817364 </td>
<td align="right" bgcolor="#eefaee" height="23"> 30,396,723,200 
     